In [7]:
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

path='C:/Repo/MiM_Analytics_Tesis/Tesis/'

In [14]:
postgreSQL = {"host": "localhost", "port": "5432", "username": "postgres", 
               "password": "trivisono", "database": "SportsAnalytics"}

uri = 'postgresql+psycopg2://'+postgreSQL['username']+':'+postgreSQL['password']+'@'+postgreSQL['host']+':'+ postgreSQL['port']+'/'+postgreSQL['database']

engine = create_engine(uri)
conn = engine.raw_connection()
cursor = conn.cursor()

In [15]:
df = pd.read_sql_query("SELECT * FROM fdm.tesis_abt;", conn)

In [16]:
df.shape

(7244, 118)

In [17]:
df.isna().sum() / len(df)

country_name                    0.000000
country_code                    0.000000
league_id                       0.000000
league_name                     0.000000
league_type                     0.000000
league_season                   0.000000
league_round                    0.000000
league_round_number             0.000000
league_progress                 0.000000
total_rounds                    0.000000
total_teams                     0.000000
fixture_id                      0.000000
referee                         0.000000
date_tz                         0.000000
date_match                      0.000000
date_dow                        0.000000
time_match                      0.000000
date_day                        0.000000
venue_id                        0.000000
teams_home_id                   0.000000
teams_home_name                 0.000000
teams_away_id                   0.000000
teams_away_name                 0.000000
target                          0.000000
rank_last       

In [57]:
df[df['date_dow'].isna()]

,country_name,country_code,league_id,league_name,league_type,league_season,league_round,league_round_number,league_progress,total_rounds,total_teams,fixture_id,referee,date_tz,date_match,date_dow,time_match,date_day,venue_id,teams_home_id,teams_home_name,teams_away_id,teams_away_name,target,rank_last,rank_points_last,rank_points_leader,rank_points_top5,home_rank,home_rank_points,home_rank_goals_diff,away_rank,away_rank_points,away_rank_goals_diff,home_formation,away_formation,home_form,home_points_won_l5,home_ha_form,home_ha_points_won_l5,away_form,away_points_won_l5,away_ha_form,away_ha_points_won_l5,home_gk_rating_l5,home_df_rating_l5,home_mf_rating_l5,home_fw_rating_l5,home_team_rating_l5,away_gk_rating_l5,away_df_rating_l5,away_mf_rating_l5,away_fw_rating_l5,away_team_rating_l5,home_matches_cnt,away_matches_cnt,home_shots_on_goal_l5,home_shots_off_goal_l5,home_total_shots_l5,home_blocked_shots_l5,home_shots_insidebox_l5,home_shots_outsidebox_l5,home_fouls_l5,home_corner_kicks_l5,home_offsides_l5,home_ball_possesion_l5,home_yellow_cards_l5,home_red_cards_l5,home_gk_saves_l5,home_total_passes_l5,home_passes_acc_l5,home_passes_pct_l5,home_fouls_opp_l5,home_yellow_cards_opp_l5,home_red_cards_opp_l5,home_total_shots_opp_l5,home_shots_on_goal_opp_l5,home_shots_off_goal_opp_l5,home_shots_outsidebox_opp_l5,home_blocked_shots_opp_l5,home_corner_kicks_opp_l5,away_shots_on_goal_l5,away_shots_off_goal_l5,away_total_shots_l5,away_blocked_shots_l5,away_shots_insidebox_l5,away_shots_outsidebox_l5,away_fouls_l5,away_corner_kicks_l5,away_offsides_l5,away_ball_possesion_l5,away_yellow_cards_l5,away_red_cards_l5,away_gk_saves_l5,away_total_passes_l5,away_passes_acc_l5,away_passes_pct_l5,away_fouls_opp_l5,away_yellow_cards_opp_l5,away_red_cards_opp_l5,away_total_shots_opp_l5,away_shots_on_goal_opp_l5,away_shots_off_goal_opp_l5,away_shots_outsidebox_opp_l5,away_blocked_shots_opp_l5,away_corner_kicks_opp_l5,home_max_scorer,away_max_scorer,home_max_assister,away_max_assister,home_max_saver,away_max_saver,home_motivation_cup,home_motivation_leader,home_motivation_stay,away_motivation_cup,away_motivation_leader,away_motivation_stay
1156,Brazil,BR,71,Serie A,League,2020,Regular Season - 10,10,0.263158,38,20,328081,"Vinicius Gomes do Amara, Brazil",2020-09-13T21:00:00+00:00,None,None,None,Evening,11287,118,Bahia,144,Atletico Goianiense,Away,11,5,20,12.0,5,12,-3,7,10,-3,4-1-4-1,4-2-3-1,LDLDL,0.133333,LDLDL,LLDWW,0.333333,0.333333,WDLLD,0.333333,7.050000,6.310000,6.202000,6.800000,6.372727,6.760000,6.838750,6.280000,7.350000,6.624091,5.0,5.0,4.0,5.0,13.0,3.0,6.0,6.0,10.0,7.0,1.0,0.480,1.0,0.0,3.0,439.0,365.0,0.824,14.0,2.0,0.0,13.0,5.0,5.0,8.0,2.0,5.0,4.0,7.0,13.0,2.0,5.0,7.0,17.0,6.0,1.0,0.544,1.0,0.0,2.0,440.0,364.0,0.828,15.0,3.0,0.0,8.0,3.0,3.0,5.0,2.0,2.0,5,5,2,2,18,8,0,0,0,0,0,0
1181,Brazil,BR,71,Serie A,League,2020,Regular Season - 13,13,0.342105,38,20,328111,"Rafael Traci, Brazil",2020-10-04T21:15:00+00:00,None,None,None,Evening,11287,118,Bahia,123,Sport Recife,Away,12,9,24,19.0,10,12,-6,6,17,-1,4-4-2,4-2-3-1,LLLLD,0.066667,LLLLD,LLLDW,0.333333,0.666667,DLWLD,0.333333,6.600000,5.700000,6.892500,6.180000,6.302727,7.200000,6.660000,5.888000,6.560000,6.349091,5.0,5.0,4.0,6.0,14.0,3.0,8.0,6.0,12.0,7.0,3.0,0.504,1.0,0.0,2.0,430.0,358.0,0.822,15.0,3.0,0.0,12.0,4.0,5.0,7.0,2.0,4.0,3.0,3.0,9.0,3.0,5.0,4.0,13.0,4.0,1.0,0.420,1.0,0.0,3.0,379.0,303.0,0.790,16.0,2.0,0.0,15.0,4.0,7.0,9.0,3.0,6.0,7,5,3,4,17,11,0,0,0,0,0,0
1196,Brazil,BR,71,Serie A,League,2020,Regular Season - 14,14,0.368421,38,20,328121,"Jean Pierre Goncalves Lima, Brazil",2020-10-07T22:15:00+00:00,None,None,None,Evening,11287,118,Bahia,133,Vasco DA Gama,Home,12,9,27,20.0,11,12,-7,6,19,2,4-2-3-1,4-2-3-1,LLLLL,0.000000,LLLLL,LLLLD,0.266667,0.266667,LLWDL,0.266667,6.440000,5.415000,6.636000,6.820000,6.190909,6.900000,6.415000,6.136000,6.840000,6.370909,5.0,5.0,6.0,7.0,17.0,4.0,10.0,7.0,13.0,8.0,3.0,0.544,1.0,0.0,2.0,463.0,393.0,0.848,15.0,3.0,0.0,11.0,4.0,5.0,7.0,2.0,2.0,3.0,3.0,10.0,3.0,4.0,

In [18]:
df.to_csv(path+'tesis_data.csv', encoding='latin-1', decimal='.', sep='|', index=False)

In [19]:
df = pd.read_csv(path+'tesis_data.csv', encoding='latin-1', decimal='.', sep='|')

In [20]:
df.head()

,country_name,country_code,league_id,league_name,league_type,league_season,league_round,league_round_number,league_progress,total_rounds,total_teams,fixture_id,referee,date_tz,date_match,date_dow,time_match,date_day,venue_id,teams_home_id,teams_home_name,teams_away_id,teams_away_name,target,rank_last,rank_points_last,rank_points_leader,rank_points_top5,home_rank,home_rank_points,home_rank_goals_diff,away_rank,away_rank_points,away_rank_goals_diff,home_formation,away_formation,home_form,home_points_won_l5,home_ha_form,home_ha_points_won_l5,away_form,away_points_won_l5,away_ha_form,away_ha_points_won_l5,home_gk_rating_l5,home_df_rating_l5,home_mf_rating_l5,home_fw_rating_l5,home_team_rating_l5,away_gk_rating_l5,away_df_rating_l5,away_mf_rating_l5,away_fw_rating_l5,away_team_rating_l5,home_matches_cnt,away_matches_cnt,home_shots_on_goal_l5,home_shots_off_goal_l5,home_total_shots_l5,home_blocked_shots_l5,home_shots_insidebox_l5,home_shots_outsidebox_l5,home_fouls_l5,home_corner_kicks_l5,home_offsides_l5,home_ball_possesion_l5,home_yellow_cards_l5,home_red_cards_l5,home_gk_saves_l5,home_total_passes_l5,home_passes_acc_l5,home_passes_pct_l5,home_fouls_opp_l5,home_yellow_cards_opp_l5,home_red_cards_opp_l5,home_total_shots_opp_l5,home_shots_on_goal_opp_l5,home_shots_off_goal_opp_l5,home_shots_outsidebox_opp_l5,home_blocked_shots_opp_l5,home_corner_kicks_opp_l5,away_shots_on_goal_l5,away_shots_off_goal_l5,away_total_shots_l5,away_blocked_shots_l5,away_shots_insidebox_l5,away_shots_outsidebox_l5,away_fouls_l5,away_corner_kicks_l5,away_offsides_l5,away_ball_possesion_l5,away_yellow_cards_l5,away_red_cards_l5,away_gk_saves_l5,away_total_passes_l5,away_passes_acc_l5,away_passes_pct_l5,away_fouls_opp_l5,away_yellow_cards_opp_l5,away_red_cards_opp_l5,away_total_shots_opp_l5,away_shots_on_goal_opp_l5,away_shots_off_goal_opp_l5,away_shots_outsidebox_opp_l5,away_blocked_shots_opp_l5,away_corner_kicks_opp_l5,home_max_scorer,away_max_scorer,home_max_assister,away_max_assister,home_max_saver,away_max_saver,home_motivation_cup,home_motivation_leader,home_motivation_stay,away_motivation_cup,away_motivation_leader,away_motivation_stay
0,Brazil,BR,71,Serie A,League,2017,Regular Season - 1,1,0.026316,38,20,35468,"Braulio Da Silva Machado, Brazil",2017-05-14T22:00:00+00:00,2017-05-14,Sunday,17:00,Afternoon,-1,130,Gremio,120,Botafogo,Home,0,0,0,NaN,0,0,0,0,0,0,4-2-3-1,4-3-2-1,NaN,-1.0,NaN,NaN,-1.0,-1.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0
1,Brazil,BR,71,Serie A,League,2017,Regular Season - 1,1,0.026316,38,20,35473,"Marcos Mateus Pereira, Brazil",2017-05-14T19:00:00+00:00,2017-05-14,Sunday,14:00,Afternoon,-1,139,Ponte Preta,123,Sport Recife,Home,0,0,0,NaN,0,0,0,0,0,0,4-2-3-1,4-2-3-1,NaN,-1.0,NaN,NaN,-1.0,-1.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0
2,Brazil,BR,71,Serie A,League,2017,Regular Season - 1,1,0.026316,38,20,35471,"Marcelo De Lima Henrique, Brazil",2017-05-14T19:00:00+00:00,2017-05-14,Sunday,14:00,Afternoon,-1,135,Cruzeiro,126,Sao Paulo,Home,0,0,0,NaN,0,0,0,0,0,0,4-2-3-1,3-4-3,NaN,-1.0,NaN,NaN,-1.0,-1.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0
3,Brazil,BR,71,Serie A,League,2017,Regular Season - 1,1,0.026316,38,20,35474,"Reway Wagner, Brazil",2017-05-14T14:00:00+00:00,2017-05-14,Sunday,09:00,Morning,-1,124,Fluminense,128,Santos,Home,0,